In [8]:
import torch
import torch.utils.data as Data
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import time
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [4]:
train_data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

In [7]:
train_data_dir = "D:/Users/hejie/Desktop/flower_images"
train_data = ImageFolder(train_data_dir,transform=train_data_transforms)
train_data_loader = Data.DataLoader(train_data,batch_size=4,shuffle=True,num_workers=1)
print("数据集的label：",train_data.targets)
for step,(b_x,b_y) in enumerate(train_data_loader):
    if step>0:
        break
print(b_x.shape)
print(b_y.shape)
print("图像的取值范围为：",b_x.min(),"~",b_x.max())

数据集的label： [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [11]:
class MyConvNet(nn.Module):
    def __init__(self):
        super(MyConvNet,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = 3,
                out_channels=16,
                kernel_size= 3,
                stride = 1,
                padding=1,
            ),
        nn.ReLU(),
            nn.AvgPool2d(
                kernel_size = 2,
                stride=2,
            ),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,3,1,0),
            nn.ReLU(),
            nn.AvgPool2d(2,2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(32*55*55,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,5)
        )
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1)
        output = self.classifier(x)
        return output
myconvnet = MyConvNet()
print(myconvnet)

MyConvNet(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (classifier): Sequential(
    (0): Linear(in_features=96800, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=5, bias=True)
  )
)


In [12]:
def train_model(model,traindataloader,train_rate,criterion,optimizer,num_epochs=25):
   batch_num = len(traindataloader)
   train_batch_num = round(batch_num*train_rate)
   best_model_wts = copy.deepcopy(model.state_dict())
   best_acc = 0.0
   train_loss_all = []
   train_acc_all = []
   val_loss_all = []
   val_acc_all = []
   since = time.time()
   for epoch in range(num_epochs):
      print('Epoch{}/{}'.format(epoch,num_epochs - 1))
      print('-' * 10)
      train_loss = 0.0
      train_corrects = 0
      train_num = 0
      val_loss = 0
      val_corrects = 0
      val_num = 0
      for step,(b_x,b_y) in enumerate(traindataloader):
        if step < train_batch_num:
            model.train()
            output = model(b_x)
            pre_lab = torch.argmax(output,1)
            loss = criterion(output,b_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * b_x.size(0)
            train_corrects += torch.sum(pre_lab == b_y.data)
            train_num += b_x.size(0)
        else:
            model.eval()
            output = model(b_x)
            pre_lab = torch.argmax(output,1)
            loss = criterion(output,b_y)
            val_loss += loss.item() * b_x.size(0)
            val_corrects += torch.sum(pre_lab == b_y.data)
            val_num += b_x.size(0)
      train_loss_all.append(train_loss / train_num)
      train_acc_all.append(train_corrects.double().item()/train_num)
      val_loss_all.append(val_loss / val_num)
      val_acc_all.append(val_corrects.double().item()/val_num)
      print('{}Train Loss:{:.4f}  Train Acc: {:.4f}'.format(epoch,train_loss_all[-1],train_acc_all[-1]))
      print('{}Val Loss:{:.4f}  Val Acc: {:.4f}'.format(epoch,val_loss_all[-1],val_acc_all[-1]))
      if val_acc_all[-1] > best_acc:
         best_acc = val_acc_all[-1]
         best_model_wts = copy.deepcopy(model.state_dict())
      time_use = time.time() - since
      print("Train and val complete in {:.0f}m {:.0f}s".format(time_use // 60,time_use % 60))
   model.load_state_dict(best_model_wts)
   train_process = pd.DataFrame(
      data={"epoch":range(num_epochs),
            "train_loss_all":train_loss_all,
            "val_loss_all":val_loss_all,
            "train_acc_all":train_acc_all,
            "val_acc_all":val_acc_all
            })
   return model,train_process

In [14]:
optimizer = torch.optim.Adam(myconvnet.parameters(),lr=0.0003)
criterion = nn.CrossEntropyLoss()
myconvnet,train_process = train_model(
    myconvnet,train_data_loader,0.8,criterion,optimizer,num_epochs=25)

Epoch0/24
----------
0Train Loss:1.2991  Train Acc: 0.4728
0Val Loss:1.2286  Val Acc: 0.4945
Train and val complete in 4m 29s
Epoch1/24
----------
1Train Loss:1.1727  Train Acc: 0.5430
1Val Loss:1.1665  Val Acc: 0.5365
Train and val complete in 10m 12s
Epoch2/24
----------
2Train Loss:1.1025  Train Acc: 0.5653
2Val Loss:1.0617  Val Acc: 0.6096
Train and val complete in 16m 4s
Epoch3/24
----------
3Train Loss:1.0499  Train Acc: 0.5880
3Val Loss:0.9831  Val Acc: 0.6136
Train and val complete in 21m 54s
Epoch4/24
----------
4Train Loss:1.0009  Train Acc: 0.6128
4Val Loss:0.9618  Val Acc: 0.6517
Train and val complete in 28m 3s
Epoch5/24
----------
5Train Loss:0.9568  Train Acc: 0.6272
5Val Loss:0.9894  Val Acc: 0.6026
Train and val complete in 34m 8s
Epoch6/24
----------
6Train Loss:0.9376  Train Acc: 0.6385
6Val Loss:0.9566  Val Acc: 0.6086
Train and val complete in 40m 14s
Epoch7/24
----------
7Train Loss:0.9236  Train Acc: 0.6470
7Val Loss:0.8968  Val Acc: 0.6537
Train and val complete